In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import scipy.sparse as sparse
import traceback
import random
from sklearn import metrics
import re
import implicit
import json
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import ParameterGrid

from implicit.evaluation import train_test_split, ranking_metrics_at_k
print(implicit.__version__)
from als_tuner import fine_tune_als


0.7.2


In [2]:
#        params = {"factors": factors,"iterations": iterations,"use_native":use_native,"use_cg":use_cg}


factors                 =         40
iterations              =         40
alpha_val               =         40
K                       =         10
use_native              =         True
use_cg                  =         True
use_gpu                 =         False
scenario                =         'upsell'
n_similars              =         5
split_ratio             =         0.3
compute_aucscore        =         True
learning_rate           =         0.01
num_threads             =         0
return_score            =         True

In [3]:
data = pd.read_csv('preprocessed_movie_rec_data.csv')
data

,User,Movie_title,rating,genre
0,1,Toy Story (1995),4.0,"Fantasy, Children, Adventure, Animation, Comedy"
1,5,Toy Story (1995),4.0,"Fantasy, Children, Adventure, Animation, Comedy"
2,7,Toy Story (1995),4.5,"Fantasy, Children, Adventure, Animation, Comedy"
3,15,Toy Story (1995),2.5,"Fantasy, Children, Adventure, Animation, Comedy"
4,17,Toy Story (1995),4.5,"Fantasy, Children, Adventure, Animation, Comedy"
...,...,...,...,...
100831,578,"Young Victoria, The (2009)",4.5,"Drama, Romance"
100832,175,Raise Your Voice (2004),5.0,Romance
100833,578,Cold Creek Manor (2003),2.5,"Drama, Thriller"
100834,578,My Blueberry Nights (2007),4.0,"Drama, Romance"


In [4]:
print(f"number of users:{data.User.nunique()}")
print(f"number of Moves:{data.Movie_title.nunique()}")

number of users:610
number of Moves:9719


In [5]:
####################preprocessing#########################
genre_dict = dict(zip(data.Movie_title, data.genre))



data['User'] = data['User'].astype(str)
data['Movie_title'] = data['Movie_title'].astype(str)
data['Movie_title'] = data['Movie_title'].str.strip()

#get category codes
data['User'] = data['User'].astype("category")
data['Movie_title'] = data['Movie_title'].astype("category")
#creating unique ID's for users and items
data['user_id'] = data['User'].cat.codes
data['item_id'] = data['Movie_title'].cat.codes
        
# creating Mappings and storing into dictionaries............
userID_to_UID = dict(enumerate(data['User'].cat.categories))
UID_to_userID = dict(map(reversed,userID_to_UID.items()))
        
itemID_to_ItemNumber = dict(enumerate(data['Movie_title'].cat.categories))
ItemNumber_to_itemID = dict(map(reversed,itemID_to_ItemNumber.items())) 

        
#creating sparse matrices.....
# The implicit library expects data as a item-user matrix
sparse_item_user = sparse.csr_matrix((data['rating'].astype(float), (data['user_id'], data['item_id'])))

#model = AlternatingLeastSquares(factors=factors , iterations=iterations ,use_native=use_native,use_cg=use_cg)

In [6]:
# def evaluate_model(model, train,test, K=10):
    
#     score = implicit.evaluation.AUC_at_k(model, train, test, K=K, show_progress=True)
#     # Dummy evaluation: Normally you'd use metrics like MAP@K, NDCG@K, Recall@K, etc.
#     # This just returns a random score for illustration.
#     return score

# def fine_tune_als(user_items,
#                   factors_list, 
#                   iterations_list, 
#                   alpha_list, 
#                   K_list,
#                   search_type='random', 
#                   n_iter=10, 
#                   random_state=42):
    
#     random.seed(random_state)

#     param_grid = {
#         'factors': factors_list,
#         'iterations': iterations_list,
#         'alpha_val': alpha_list,
#         'K': K_list
#     }

#     all_params = list(ParameterGrid(param_grid))

#     if search_type == 'random':
#         sampled_params = random.sample(all_params, min(n_iter, len(all_params)))
#     elif search_type == 'grid':
#         sampled_params = all_params
#     else:
#         raise ValueError("search_type must be 'random' or 'grid'")

#     best_score = -np.inf
#     best_params = None

#     for params in sampled_params:
#         model = AlternatingLeastSquares(factors=params['factors'], 
#                                         iterations=params['iterations'])
#         train, test = train_test_split(user_items, train_percentage = split_ratio)
#         # Weighting by alpha_val
#         weighted_user_items = (train * params['alpha_val']).astype('double')

#         model.fit(train)

#         score = evaluate_model(model, train, test, K=params['K'])

#         if score > best_score:
#             best_score = score
#             best_params = params

#     return best_params, best_score

# # Example Usage
# # user_items = ... # your sparse user-item matrix



# tuned_params, tuned_score = fine_tune_als(
#     user_items = sparse_item_user,
#     factors_list=[x for x in range(20, 200, 5)],
#     iterations_list=[x for x in range(20, 200, 10)],
#     alpha_list=[10, 20, 40],
#     K_list=[x for x in range(20, 100, 10)],
#     search_type='random',
#     n_iter=5
# )

# print("Best Parameters:", tuned_params)
# print("Best Score:", tuned_score)

In [7]:
tuned_params, tuned_score = fine_tune_als(
    user_items = sparse_item_user,
    factors_list=[x for x in range(20, 200, 5)],
    iterations_list=[x for x in range(20, 200, 10)],
    alpha_list=[10, 20, 40],
    K_list=[x for x in range(20, 100, 10)],
    search_type='random',
    n_iter=5
)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

In [8]:
print(tuned_params, tuned_score)

{'K': 70, 'alpha_val': 20, 'factors': 50, 'iterations': 20} 0.6029468454589336


In [9]:
sparse_item_user.get_shape()

(610, 9719)

In [10]:
keys_to_remove = {'K', 'alpha_val'}

final_params = {k: v for k, v in tuned_params.items() if k not in keys_to_remove}
final_params

{'factors': 50, 'iterations': 20}

In [12]:
model = AlternatingLeastSquares(**final_params)
weighted_user_items = sparse_item_user*alpha_val
model.fit(weighted_user_items)

  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
model.save('implicit_model.npz')

In [18]:
model.load('implicit_model.npz')

In [15]:
import pickle
with open('recommendation_model.pkl', 'wb') as out:
            pickle.dump(model, out)

In [19]:
if compute_aucscore:
    # splitting data into train and test....    
    train, test = train_test_split(sparse_item_user, train_percentage = split_ratio)       
    print('Training implicit model')
    #ALS Model.....
    weighted_user_items = (sparse_item_user * tuned_params['alpha_val']).astype('double')
    model.fit(sparse_item_user * alpha_val)
    item_vecs = model.item_factors
    user_vecs = model.user_factors
    metrics = ranking_metrics_at_k(model, train, test, K=K,show_progress=True, num_threads=num_threads)
    print('Training complete.')
    scores = [str(key)+'_'+str(value) for key,value in metrics.items()]
    all_metrics = '-'.join(x for x in scores)   
else:
    model.fit(sparse_item_user.T * alpha_val)
    all_metrics = None
    print('Training complete.')


Training implicit model


  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

Training complete.


In [11]:
all_metrics

'precision_0.21147540983606558-map_0.11936293260473588-ndcg_0.1901549518521156-auc_0.5450899288620303'

In [12]:
item_vecs = model.item_factors
user_vecs = model.user_factors
print(item_vecs.shape,user_vecs.shape)

(9719, 40) (610, 40)


In [13]:
def cross_sell(CID,sparse_matrix,n_similars,return_score):
            cross_cell_list = []
            cross_sell_items = model.recommend(UID_to_userID[CID], sparse_matrix.T,N = n_similars,filter_already_liked_items=True)
            for item in cross_sell_items:
                idx, score = item
                if return_score == True:
                    cross_cell_list.append(str(itemID_to_ItemNumber[idx]) + "," + str(score))
                if return_score == False:
                    cross_cell_list.append(str(itemID_to_ItemNumber[idx]))
            return cross_cell_list[:n_similars]
            
#function to return recommended items for up_sell scenario
def up_sell(IID,n_similars,return_score):
    up_cell_list = []
    upsell_items = model.similar_items(ItemNumber_to_itemID[IID], n_similars)
    temp_list = [item_tuple for item_tuple in similar_users if UID_to_userID[CID]!=item_tuple[0]]
    for item in temp_list:
        idx, score = item
        if return_score == True:
            up_cell_list.append(str(itemID_to_ItemNumber[idx]) + "," + str(score))
        if return_score == False:
            up_cell_list.append(str(itemID_to_ItemNumber[idx]))
    return up_cell_list[:n_similars]

#function to return similar users for a user scenario
def sim_users(CID,n_similars,return_score):
    sim_users_list = []
    similar_users = model.similar_users(UID_to_userID[CID], n_similars+5)
    for user in similar_users:
        idx, score = user
        if return_score == True:
            sim_users_list.append(str(userID_to_UID[idx]) + "," + str(score))
        if return_score == False:
            sim_users_list.append(str(userID_to_UID[idx]))
    return sim_users_list[:n_similars]

#function to rank items for a given user. 
def rank_the_items(CID,sparse_matrix,item_lst,return_score):
    ranked_items = []
    if len(item_lst)==0:
        ranked_items.append('no__similar__items')
    else:    
        cross_sell_items = model.rank_items(UID_to_userID[CID], sparse_matrix.T,item_lst)
        #temp_list = [item_tuple for item_tuple in cross_sell_items if UID_to_userID[CID]!=item_tuple[0]]
        for item in cross_sell_items:
            idx, score = item
            if return_score == True:
                ranked_items.append(str(itemID_to_ItemNumber[idx]) + "," + str(score))
            if return_score == False:
                ranked_items.append(str(itemID_to_ItemNumber[idx]))
    return ranked_items 

In [14]:
user = 1
item = 'Little Ashes (2008)'

In [15]:
explaination = model.explain(int(userID_to_UID[user]), sparse_item_user, ItemNumber_to_itemID[item], user_weights=None, N=5)
explaination

(0.059775773856051255,
 [(2718, 0.011321517003243218),
  (8722, 0.007764773842487329),
  (4433, 0.006925168762858794),
  (191, 0.005570294008169316),
  (4701, 0.00533203770169138)],
 (array([[ 0.58171603, -0.04636782, -0.08311552, ..., -0.04931596,
          -0.03605708,  0.04398253],
         [-0.0269729 ,  0.73183668,  0.24858655, ..., -0.05930919,
           0.09041805,  0.12909893],
         [-0.04834963,  0.18577864,  0.70265555, ..., -0.02192623,
           0.03350223,  0.05165326],
         ...,
         [-0.02868788, -0.04111797, -0.02605114, ...,  0.51874216,
           0.00198335,  0.03673074],
         [-0.02097498,  0.06784313,  0.04901415, ...,  0.00554083,
           0.53853498, -0.00482693],
         [ 0.02558534,  0.09243996,  0.06473107, ...,  0.0072433 ,
           0.03519173,  0.53814896]]),
  False))

In [20]:
def get_similar_items(itemid,n_similars):
    similar_items = []
    ids,scores = model.similar_items(ItemNumber_to_itemID[itemid],n_similars,filter_items = [ItemNumber_to_itemID[itemid]])
    for x,y in zip(ids,scores):
        similar_items.append(itemID_to_ItemNumber[x])
    return similar_items

def get_recommendations(userid,n_similars,sparse_item_user):# sparse_item_user
    recommended_items = []
    user_interactions = sparse_item_user.tocsr()
    ids,scores = model.recommend(int(userID_to_UID[userid]),user_interactions.getrow(userid), N = n_similars,filter_already_liked_items=True)#sparse_item_user[userID_to_UID[userid]]
    for x,y in zip(ids,scores):
        recommended_items.append(itemID_to_ItemNumber[x])
    return recommended_items

In [21]:
get_similar_items(item,5)

['Cold Creek Manor (2003)',
 'My Blueberry Nights (2007)',
 'Cheaper by the Dozen (1950)',
 'Young Victoria, The (2009)',
 'Letters to Juliet (2010)']

In [22]:
userID_to_UID[user]
#sparse_item_user[userID_to_UID[user]]
print(type(userID_to_UID[user]))

<class 'str'>


In [23]:
get_recommendations(user,5,sparse_item_user)

['Say Anything... (1989)',
 'Dancer in the Dark (2000)',
 'Lost in Translation (2003)',
 "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
 'Toy Story 2 (1999)']

In [104]:
UID_to_userID[user], IId_to_ItemID

KeyError: 1

In [105]:
ItemNumber_to_itemID[item], itemID_to_ItemNumber[5108]

(5108, 'Little Ashes (2008)')

In [106]:
ids,scores = model.similar_items(5108,5)
print(itemID_to_ItemNumber[5108])
print(ids,scores)

Little Ashes (2008)
[5108 1877 5928 1700 9662] [1.         0.99999225 0.9999663  0.92081267 0.8451694 ]


In [107]:
for x,y in zip(ids,scores):
    print(itemID_to_ItemNumber[x],y)

Little Ashes (2008) 1.0
Cold Creek Manor (2003) 0.99999225
My Blueberry Nights (2007) 0.9999663
Cheaper by the Dozen (1950) 0.92081267
Young Victoria, The (2009) 0.8451694


In [108]:
temp_list = [item_tuple for item_tuple in items if 5!=item_tuple[0]]

temp_list

NameError: name 'items' is not defined

In [109]:
up_cell_list = []
for item in temp_list:
    print(item)
    idx, score = item
    if return_score == True:
        up_cell_list.append(str(itemID_to_ItemNumber[idx]) + "," + str(score))
    if return_score == False:
        up_cell_list.append(str(itemID_to_ItemNumber[idx]))

NameError: name 'temp_list' is not defined

In [13]:
#c_list = cross_sell(user,sparse_item_user,n_similars,return_score)
u_list = up_sell(item,n_similars,return_score)

IndexError: index 5108 is out of bounds for axis 0 with size 610

In [ ]:
print(c_list,u_list)